<a href="https://colab.research.google.com/github/giagia96/NLP---Assignement-4/blob/master/SQUAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install tokenizers
!pip install transformers

In [27]:
from keras.layers import Input, TimeDistributed, LSTM, Bidirectional, Dense, Activation, Multiply, Add, Lambda
import pandas as pd
import json
import numpy as np
from keras.layers import *
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
import string 

In [28]:
with open('/content/drive/MyDrive/training_set.json') as f:
    raw_train_data = json.load(f)

In [33]:
word2idx = {}
idx2word = {}
max_len_question = 0
max_len_context = 0
max_len_answer = 0
idx = 1

for item in raw_train_data["data"]:
  for para in item["paragraphs"]:
    context = para["context"]
    for qa in para["qas"]:
      question = qa["question"]
      answer = qa["answers"][0]["text"]
      text = [context, question, answer]
      for txt in text:
        for x in txt.split():
          if '–' in x or '-' in x:
            x = x.replace('–', ' ').replace('-', ' ').split()
            for i in x:
              i = re.sub(r'[^A-Za-z0-9]+', '', i)
              if i not in word2idx.keys():
                word2idx[i] = idx
                idx2word[idx] = i
                idx += 1
          else:
            x = re.sub(r'[^A-Za-z0-9]+', '', x)
            if x not in word2idx.keys():
              word2idx[str(x)] = idx
              idx2word[idx] = x
              idx += 1
print(len(word2idx))
print(idx)

102666
102667


In [34]:
def tokenize(text):
  sentence = []
  for y in text.split():
    if '–' in y or '-' in y:
      y = y.replace('–', ' ').replace('-', ' ').split()
      for i in y:
        i = re.sub(r'[^A-Za-z0-9]+', '', i)
        sentence.append(word2idx[i])
    else:
      y = re.sub(r'[^A-Za-z0-9]+', '', y)
      sentence.append(word2idx[y])
  return sentence
  
tokenized_context = []
tokenized_question = []
tokenized_answer = []
unt_context = []
unt_answer = []
unt_question = []
for item in raw_train_data["data"]:
  for para in item["paragraphs"]:
    context = para["context"]
    for qa in para["qas"]:
      question = qa["question"]
      answers = qa["answers"]
      for answer in answers:
        answer1 = answer['text']
        unt_context.append(context)
        unt_answer.append(answer1)
        unt_question.append(question)
        tokenized_context.append(tokenize(context))
        tokenized_answer.append(tokenize(answer1))
        tokenized_question.append(tokenize(question))
print(len(tokenized_context))
print(len(tokenized_answer))
print(len(tokenized_question))

87599
87599
87599


In [92]:
from keras.preprocessing.sequence import pad_sequences

attention_mask = []
input_ids = []
token_type_ids = []
start_tokens = []
end_tokens = []
max_len = 0
for x in range(len(tokenized_context)):  
  input_id = tokenized_context[x] + tokenized_question[x]
  toks = [(i, i+len(tokenized_answer[x])) for i in range(len(tokenized_context[x])) if tokenized_context[x][i:i+len(tokenized_answer[x])] == tokenized_answer[x]]
  if len(toks) != 0:
    start_token = toks[0][0]
    end_token = toks[0][1]
    start_tokens.append(start_token)
    end_tokens.append(end_token)
    attention = [1] * len(input_id)
    padding_length = max_len - len(input_id)
    token_ids = [0] * len(tokenized_context[x]) + [1] * len(
              tokenized_question[x][0:]
          )
    if len(input_id) > max_len:
      max_len = len(input_id)
    input_ids.append(input_id)
    attention_mask.append(attention)
    token_type_ids.append(token_ids)

input_ids_train = pad_sequences(input_ids[101:], maxlen= max_len, padding='post', truncating = 'post')
attention_mask_train = pad_sequences(attention_mask[101:], maxlen= max_len, padding='post', truncating = 'post')
token_type_ids_train = pad_sequences(token_type_ids[101:], maxlen= max_len, padding='post', truncating = 'post')

input_ids_test = pad_sequences(input_ids[51:100], maxlen= max_len, padding='post', truncating = 'post')
attention_mask_test = pad_sequences(attention_mask[51:100], maxlen= max_len, padding='post', truncating = 'post')
token_type_ids_test = pad_sequences(token_type_ids[51:100], maxlen= max_len, padding='post', truncating = 'post')

input_ids_val = pad_sequences(input_ids[:50], maxlen= max_len, padding='post', truncating = 'post')
attention_mask_val = pad_sequences(attention_mask[:50], maxlen= max_len, padding='post', truncating = 'post')
token_type_ids_val = pad_sequences(token_type_ids[:50], maxlen= max_len, padding='post', truncating = 'post')

start_tokens_train = start_tokens[101:]
end_tokens_train = end_tokens[101:]

start_tokens_test = start_tokens[51:100]
end_tokens_test = end_tokens[51:100]

start_tokens_val = start_tokens[:50]
end_tokens_val = end_tokens[:50]
print(len(start_tokens_train))

x_train = [
     input_ids_train,
     token_type_ids_train,
     attention_mask_train
]
y_train = [
     np.array(start_tokens_train),
     np.array(end_tokens_train)
]

x_test = [
     input_ids_test,
     token_type_ids_test,
     attention_mask_test
]

y_test = [
     np.array(start_tokens_test),
     np.array(end_tokens_test)
]

x_val = [
     input_ids_val,
     token_type_ids_val,
     attention_mask_val
]
y_val = [
     np.array(start_tokens_val),
     np.array(end_tokens_val)
]

86212


In [96]:
configuration = BertConfig() 
configuration.vocab_size = len(word2idx.keys())
configuration.hidden_size = 756
configuration.max_position_embeddings = 512
configuration.num_attention_heads = 3
configuration.num_hidden_layers = 3
print(configuration)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 756,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 3,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "type_vocab_size": 2,
  "vocab_size": 102666
}



In [97]:
def create_model():   
    encoder =TFBertModel(config=configuration)

    ## QA Model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]

    start_logits = layers.Dense(1, name="start_logit", use_bias=False)(embedding)
    start_logits = layers.Flatten()(start_logits)

    end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
    end_logits = layers.Flatten()(end_logits)

    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss], metrics=['accuracy'])
    return model

In [98]:
def normalize_text(text):
    text = text.lower()

    # Remove punctuations
    exclude = set(string.punctuation)
    text = "".join(ch for ch in text if ch not in exclude)

    # Remove articles
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(regex, " ", text)

    # Remove extra white space
    text = " ".join(text.split())
    return text


class ExactMatch(keras.callbacks.Callback):
    """
    Each `SquadExample` object contains the character level offsets for each token
    in its input paragraph. We use them to get back the span of text corresponding
    to the tokens between our predicted start and end tokens.
    All the ground-truth answers are also present in each `SquadExample` object.
    We calculate the percentage of data points where the span of text obtained
    from model predictions matches one of the ground-truth answers.
    """

    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]

            normalized_pred_ans = normalize_text(pred_ans)
            normalized_true_ans = [normalize_text(_) for _ in squad_eg.all_answers]
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")

In [99]:
use_tpu = True
import tensorflow as tf
if use_tpu:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
else:
    model = create_model()

model.summary()

INFO:tensorflow:Initializing the TPU system: grpc://10.121.244.34:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.121.244.34:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 666)]        0                                            
__________________________________________________________________________________________________
input_24 (InputLayer)           [(None, 666)]        0                                            
__________________________________________________________________________________________________
input_23 (InputLayer)           [(None, 666)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_9 (TFBertModel)   TFBaseModelOutputWit 99400536    input_22[0][0]                   
                                                                 input_24[0][0]             

In [100]:
model.fit(
    x_train,
    y_train,
    epochs=30,  # For demonstration, 3 epochs are recommended
    batch_size=256,
    validation_data = (x_val, y_val),
)

Epoch 1/100


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


337/337 [==============================] - ETA: 0s - loss: 9.1488 - activation_18_loss: 4.5209 - activation_19_loss: 4.6279 - activation_18_accuracy: 0.0371 - activation_19_accuracy: 0.0418

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


337/337 [==============================] - 173s 423ms/step - loss: 9.1478 - activation_18_loss: 4.5203 - activation_19_loss: 4.6274 - activation_18_accuracy: 0.0372 - activation_19_accuracy: 0.0418 - val_loss: 8.2807 - val_activation_18_loss: 3.8610 - val_activation_19_loss: 4.4198 - val_activation_18_accuracy: 0.1200 - val_activation_19_accuracy: 0.0400
Epoch 2/100
337/337 [==============================] - 116s 345ms/step - loss: 8.2098 - activation_18_loss: 3.9060 - activation_19_loss: 4.3038 - activation_18_accuracy: 0.0764 - activation_19_accuracy: 0.0568 - val_loss: 7.9257 - val_activation_18_loss: 3.6391 - val_activation_19_loss: 4.2866 - val_activation_18_accuracy: 0.1800 - val_activation_19_accuracy: 0.0600
Epoch 3/100
337/337 [==============================] - 116s 344ms/step - loss: 7.8304 - activation_18_loss: 3.6501 - activation_19_loss: 4.1802 - activation_18_accuracy: 0.1077 - activation_19_accuracy: 0.0652 - val_loss: 7.6802 - val_activation_18_loss: 3.4356 - val_activa

KeyboardInterrupt: ignored